In [472]:
import sys
sys.path.append('../../src')

In [473]:
import pandas
import numpy
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, VaryingProcess
from energiapy.components.material import Material
from energiapy.components.location import Location
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.components.transport import Transport
from energiapy.model.formulate import formulate, Constraints, Objective, ModelClass
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.plot import plot_results, plot_scenario
from energiapy.plot.plot_results import CostY, CostX
from energiapy.model.solve import solve
from itertools import product

from energiapy.components.result import Result
import pandas
from itertools import product
from matplotlib import rc
import matplotlib.pyplot as plt
from energiapy.model.solve import solve
from energiapy.plot.plot_results import CostY, CostX
from energiapy.components.location import Location
from energiapy.plot import plot_results, plot_scenario
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.bounds import CapacityBounds
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results
from energiapy.components.transport import Transport
from energiapy.components.scenario import Scenario
from energiapy.components.network import Network
from energiapy.components.location import Location
from energiapy.components.material import Material

from energiapy.components.process import Process, VaryingProcess
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.temporal_scale import TemporalScale
import numpy
from energiapy.conversion.photovoltaic import solar_power_output
from energiapy.conversion.windmill import wind_power_output
from energiapy.aggregation.reduce_scenario import reduce_scenario, Clustermethod
from energiapy.aggregation.ahc import agg_hierarchial_elbow, IncludeAHC, Fit
import matplotlib.pyplot as plt

from energiapy.model.weights import EmissionWeights


\begin{align}
\min_x \quad \theta^TH^Tx& + c^Tx\\
\text{s.t.} \quad Ax &\leq b + F\theta\\
A_{eq}x &= b_{eq}\\
A_\theta \theta &\leq b_\theta\\
x &\in R^n\\
\end{align}


In [474]:
scales = TemporalScale(discretization_list=[1])

In [475]:
bigM = 10**15
smallM = 0.0000000001

Declare Materials

In [477]:

LithiumCarbonateR = Material(name='LithiumCarbonateR', gwp=10602.89995, odp=0.0005311, acid=83.2005862, eutt=185.9710578, eutf=6.0273706, eutm=42.7091789, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label='Rock-based Lithium',
                citation='https://www.iea.org/data-and-statistics/charts/ghg-emissions-intensity-for-lithium-by-resource-type-and-processing-route')  
LithiumCarbonateB = Material(name='LithiumCarbonateB', gwp=2147.78164, odp=0.0000323, acid=14.7111022, eutt=55.7722302, eutf=1.9709617, eutm=8.3834083, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label='Brine-based Lithium',
               citation='https://www.iea.org/data-and-statistics/charts/ghg-emissions-intensity-for-lithium-by-resource-type-and-processing-route')  

Steel = Material(name ='steel', gwp= 2121.152427, odp=0.0000524, acid=7.071973, eutt=20.1478856, eutf=0.9654292, eutm=2.0671433, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Steel', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Cast_iron = Material(name ='cast iron', gwp= 1759.710573, odp=0.0000443, acid=6.0061499, eutt=18.2339132, eutf=0.66155, eutm=1.7330535, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Cast iron', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Concrete = Material(name ='concrete', gwp= 90.91, odp=0.00000040125, acid=0.246868, eutt=1.546130613, eutf=0.015425534, eutm=0.140321324, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Concrete', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Glass = Material(name = 'glass', gwp = 1118.5, odp=0.0000141, acid=8.8362743, eutt=22.0973602, eutf=0.1263297, eutm=1.8534905, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'glass',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Aluminium = Material(name = 'aluminium', gwp = 5632.53, odp=0.0000578, acid=31.2744756, eutt=65.5836037, eutf=1.8348121, eutm=6.2774955, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'aluminium',
                 citation = 'https://www.researchgate.net/figure/Overall-environmental-impact-per-kg-Pts-kg-and-GWP-kg-of-CO2-eq-kg-aluminium-of_fig2_295243557. & Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Silicon = Material(name = 'silicon', gwp = 50137.8, odp=0.0012152, acid=188.0829961, eutt=469.8237153, eutf=18.9483911, eutm=46.3346166, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'silicon',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Copper = Material(name = 'copper', gwp = 8660.4, odp=0.0003416, acid=516.3936129, eutt=409.7976693, eutf=46.0940178, eutm=29.6096174, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'copper',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')


Declare Processes

In [524]:

scales = TemporalScale(discretization_list=[1])

Solar = Resource(name='Solar', cons_max=80, basis='MW', label='Solar Power')

Wind = Resource(name='Wind', cons_max=85, basis='MW', label='Wind Power')

Power = Resource(name='Power', basis='MW', label='Power generated', demand = True, varying = [VaryingResource.UNCERTAIN_DEMAND], varying_bounds  = (0.5,1))#(min(demand['COAST']), 1))


LiI = Process(name='LiI', storage=Power, vopex=1302, capex=0, fopex=0,  prod_max=20,
            storage_loss=0.11,  label='Lithium-ion battery', basis='MW', store_max=30)

WF = Process(name='WF', conversion={Wind: -1, Power: 0.85}, vopex=990, capex=0, fopex=0,
            prod_max=95, label='Wind mill array', basis='MW', varying = [VaryingProcess.UNCERTAIN_CAPACITY]
            , varying_bounds = (0.5,1))

PV = Process(name='PV', conversion={Solar: -1, Power: 0.75}, vopex=567, capex=0, fopex=0,
            prod_max=100, label='Solar PV', basis='MW', varying = [VaryingProcess.UNCERTAIN_CAPACITY]
            , varying_bounds = (0.5,1))

region = Location(name='region', processes={LiI, WF, PV}, scales=scales, label='some region wo dispatchable')
scenario = Scenario(name='example', demand={region: {Power:100}}, network=region, scales=scales, label='example scenario')
matrix = formulate(scenario=scenario, model_class=ModelClass.MPLP)
    # print(matrix)
    # return matrix



The problem has the following variables:
Resource inventory level (Inv) x 1
Exact resource discharge (Sf) x 0
Uncertain resource discharge (S) x 1
Exact resource availability (Af) x 2
Uncertain resource availability (A) x 0
Exact resource price (Cf) x 2
Uncertain resource price (C) x 0
Exact process production (Pf) x 2
Uncertain process production (P) x 2
 For a total of 8 (5 fixed, and 3 uncertain)


In [525]:
# matrix['c'] = numpy.array([[   10],
#        [   0.],
#        [   0.],
#        [   0.],
#        [1302.],
#        [   0.],
#        [ 567.],
#        [ 990.]])

In [526]:
solution= solve(matrix = matrix, interface = 'native', name = 'results', solver = 'ppopt') 

[]


## Changes to OG problem to add MOO (emission objectives)

In [527]:
matrix['F'] = numpy.array([numpy.concatenate((i, numpy.array([0]*6)), axis = 0) for i in matrix['F']])

In [528]:
matrix['CRa'] = numpy.concatenate((numpy.eye(9),-numpy.eye(9)))

In [529]:
matrix['CRb'] = numpy.concatenate((numpy.array([[1] for i in range(9)]), numpy.array([[0] for i in range(9)])))

In [530]:
# add sum of thetas = 1
# matrix['A'] = numpy.vstack((matrix['A'], numpy.array([0]*8)))
# matrix['F'] = numpy.vstack((matrix['F'], numpy.concatenate((numpy.array([0]*3), numpy.array([-1]*6)), axis = 0)))
# matrix['b'] = numpy.vstack((matrix['b'], numpy.array([1])))

In [531]:
material_cons_LiI = {LithiumCarbonateR: 20}
material_cons_WF = {Steel: 109.9, Cast_iron: 8.805, Concrete: 398.7}
material_cons_PV = {Glass: 70, Steel: 56, Concrete: 48, Aluminium: 19, Silicon: 7, Copper: 7}
material_cons_dict = {'PV': material_cons_PV, 'WF': material_cons_WF, 'LiI': material_cons_LiI}
material_cons_dict2 = {i: {m.name: n for m,n in j.items()} for i,j in material_cons_dict.items()}
objective_dict = {i: {m.name: {a: b*material_cons_dict2[i][m.name] for a,b in m.emission_potentials_dict.items()} for m,n in j.items()} for i, j in material_cons_dict.items()}
def calculate_total_potentials(data):
    totals = {}
    for process, materials in data.items():
        totals[process] = {}
        for material, impacts in materials.items():
            for impact, value in impacts.items():
                if impact not in totals[process]:
                    totals[process][impact] = 0
                totals[process][impact] += value
    return totals

# Calculate and print total potentials
objective_dict = calculate_total_potentials(objective_dict)
H1 = numpy.zeros((3,8))
LiI_obj = numpy.array([[objective_dict['LiI'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
LiI_d_obj = numpy.array([[0] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
PV_obj = numpy.array([[objective_dict['PV'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
WF_obj = numpy.array([[objective_dict['WF'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
H2 = numpy.concatenate((numpy.zeros((6,4)), numpy.concatenate((LiI_obj, LiI_d_obj, PV_obj, WF_obj), axis = 1)), axis = 1)
matrix['H'] = numpy.concatenate((H1, H2)).transpose()

In [532]:
scenario.A_df

,Inv_LiI_Power_stored,S_Power,Af_Solar,Af_Wind,Pf_LiI,Pf_LiI_discharge,P_PV,P_WF
MB_LiI_Power_stored(=),1.0,0.0,0.0,0.0,1.0,-1.00,0.00,0.00
MB_Power(=),0.0,-1.0,0.0,0.0,-1.0,0.89,0.75,0.85
MB_Solar(=),0.0,0.0,1.0,0.0,0.0,0.00,-1.00,0.00
MB_Wind(=),0.0,0.0,0.0,1.0,0.0,0.00,0.00,-1.00
Inv_LiI_Power_stored(<),1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
S_Power(>),0.0,-1.0,0.0,0.0,0.0,0.00,0.00,0.00
Af_Solar(<),0.0,0.0,1.0,0.0,0.0,0.00,0.00,0.00
Af_Wind(<),0.0,0.0,0.0,1.0,0.0,0.00,0.00,0.00
Pf_LiI(<),0.0,0.0,0.0,0.0,1.0,0.00,0.00,0.00
Pf_LiI_discharge(<),0.0,0.0,0.0,0.0,0.0,1.00,0.00,0.00


In [533]:
scenario.F_df

,Th_Power,Th_PV,Th_WF
MB_LiI_Power_stored(=),0.0,0.0,0.0
MB_Power(=),0.0,0.0,0.0
MB_Solar(=),0.0,0.0,0.0
MB_Wind(=),0.0,0.0,0.0
Inv_LiI_Power_stored(<),0.0,0.0,0.0
S_Power(>),-100.0,0.0,0.0
Af_Solar(<),0.0,0.0,0.0
Af_Wind(<),0.0,0.0,0.0
Pf_LiI(<),0.0,0.0,0.0
Pf_LiI_discharge(<),0.0,0.0,0.0


In [534]:
scenario.A_df

,Inv_LiI_Power_stored,S_Power,Af_Solar,Af_Wind,Pf_LiI,Pf_LiI_discharge,P_PV,P_WF
MB_LiI_Power_stored(=),1.0,0.0,0.0,0.0,1.0,-1.00,0.00,0.00
MB_Power(=),0.0,-1.0,0.0,0.0,-1.0,0.89,0.75,0.85
MB_Solar(=),0.0,0.0,1.0,0.0,0.0,0.00,-1.00,0.00
MB_Wind(=),0.0,0.0,0.0,1.0,0.0,0.00,0.00,-1.00
Inv_LiI_Power_stored(<),1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
S_Power(>),0.0,-1.0,0.0,0.0,0.0,0.00,0.00,0.00
Af_Solar(<),0.0,0.0,1.0,0.0,0.0,0.00,0.00,0.00
Af_Wind(<),0.0,0.0,0.0,1.0,0.0,0.00,0.00,0.00
Pf_LiI(<),0.0,0.0,0.0,0.0,1.0,0.00,0.00,0.00
Pf_LiI_discharge(<),0.0,0.0,0.0,0.0,0.0,1.00,0.00,0.00


In [535]:
solution_moo = solve(matrix = matrix, interface = 'native', name = 'results', solver = 'ppopt') 

[]


In [536]:
len(solution.critical_regions)

3

In [537]:
len(solution_moo.critical_regions)

10

\begin{align}
x(\theta) &= A\theta + b\\
\lambda(\theta) &= C\theta + d\\
\Theta &:= \{\forall \theta \in \mathbf{R}^m: E\theta \leq f\}
\end{align}

In [538]:
solution.evaluate(numpy.array([[0.9] for i in range(3)]))

array([[2.00000000e+01],
       [9.00000000e+01],
       [8.00000000e+01],
       [1.43529412e+01],
       [1.90163762e-14],
       [2.00000000e+01],
       [8.00000000e+01],
       [1.43529412e+01]])

In [539]:
scenario.A_df

,Inv_LiI_Power_stored,S_Power,Af_Solar,Af_Wind,Pf_LiI,Pf_LiI_discharge,P_PV,P_WF
MB_LiI_Power_stored(=),1.0,0.0,0.0,0.0,1.0,-1.00,0.00,0.00
MB_Power(=),0.0,-1.0,0.0,0.0,-1.0,0.89,0.75,0.85
MB_Solar(=),0.0,0.0,1.0,0.0,0.0,0.00,-1.00,0.00
MB_Wind(=),0.0,0.0,0.0,1.0,0.0,0.00,0.00,-1.00
Inv_LiI_Power_stored(<),1.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
S_Power(>),0.0,-1.0,0.0,0.0,0.0,0.00,0.00,0.00
Af_Solar(<),0.0,0.0,1.0,0.0,0.0,0.00,0.00,0.00
Af_Wind(<),0.0,0.0,0.0,1.0,0.0,0.00,0.00,0.00
Pf_LiI(<),0.0,0.0,0.0,0.0,1.0,0.00,0.00,0.00
Pf_LiI_discharge(<),0.0,0.0,0.0,0.0,0.0,1.00,0.00,0.00


In [540]:
solution.evaluate_objective(numpy.array([[] for i in range(3)]))

In [541]:
c = solution.program.c
c.T@solution.evaluate(numpy.array([[0.9] for i in range(3)]))


array([[59569.41176471]])

In [542]:
solution_moo.evaluate_objective(numpy.array([[0.6] for i in range(9)]))

In [543]:
solution_moo.evaluate_objective(numpy.array([[0.8], [0.8], [0.8], [0.01], [0.01], [0.01], [0.01], [0.01], [0.01]]))

In [544]:
matrix['H'].T

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.12057999e+05, 0.00000000e+00, 7.20048686e+05, 2.84854720e+05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.06220000e-02, 0.00000000e+00, 1.59364600e-02, 6.30879987e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.66401172e+03, 0.00000000e+00, 6.55197065e+03, 9.28520254e+02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.71942116e+03, 0.00000000e+00, 1.01527492e+04, 2.

In [545]:
x_1 = numpy.linspace(0.0001,0.9999, 50)
x_2 = 1 - x_1 

for index, _ in enumerate(x_1):
    x_sol = solution_moo.evaluate(numpy.array([[0.9], [0.5], [0.6], [x_1[index]], [x_2[index]], [0.001], [0.001], [0.001], [0.001]]))
    # print(x_sol)
    if x_sol is not None:
        obj_1 = matrix['H'].T[3]@x_sol
        obj_2 = matrix['H'].T[8]@x_sol
        print(x_1[index], x_2[index], obj_1, obj_2)
        print(x_sol)
    

0.0001 0.9999 [47631209.34876828] [57337.20823094]
[[2.00000000e+01]
 [9.00000000e+01]
 [5.00000000e+01]
 [4.08235294e+01]
 [3.20156131e-14]
 [2.00000000e+01]
 [5.00000000e+01]
 [4.08235294e+01]]
0.020504081632653062 0.979495918367347 [39038260.77893287] [45606.78584244]
[[ 2.00000000e+01]
 [ 9.00000000e+01]
 [ 3.16666667e+01]
 [ 5.70000000e+01]
 [-9.44422922e-15]
 [ 2.00000000e+01]
 [ 3.16666667e+01]
 [ 5.70000000e+01]]
0.04090816326530613 0.9590918367346939 [39038260.77893287] [45606.78584244]
[[ 2.00000000e+01]
 [ 9.00000000e+01]
 [ 3.16666667e+01]
 [ 5.70000000e+01]
 [-9.44422922e-15]
 [ 2.00000000e+01]
 [ 3.16666667e+01]
 [ 5.70000000e+01]]
0.061312244897959195 0.9386877551020408 [39038260.77893287] [45606.78584244]
[[ 2.00000000e+01]
 [ 9.00000000e+01]
 [ 3.16666667e+01]
 [ 5.70000000e+01]
 [-9.44422922e-15]
 [ 2.00000000e+01]
 [ 3.16666667e+01]
 [ 5.70000000e+01]]
0.08171632653061225 0.9182836734693878 [39038260.77893287] [45606.78584244]
[[ 2.00000000e+01]
 [ 9.00000000e+01]
 [